In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, MaxPooling2D, Conv1D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.layers.wrappers import TimeDistributed
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from sklearn.metrics import classification_report, confusion_matrix

import random
import tensorflow as tf


sns.set(style='white', context='notebook', palette='deep')

from datetime import datetime

# datetime object containing current date and time
start_time = datetime.now()
import gc

Using TensorFlow backend.


In [2]:
# Movie lens 100 K dataset
import os
# data_dir="C:/Users/ebrah/Desktop/Ryerson/research1/database/movielens_100K/ml-100k/ml-100k"
names = ['user_id', 'item_id', 'rating', 'timestamp']
# movie_db = pd.read_csv(os.path.join(data_dir, 'u.data'), '\t', names=names,
#                        engine='python')
movie_db = pd.read_csv('C:/Users/ebrah/Desktop/Ryerson/research1/source_db/netflix_sample_complete.csv',skiprows=1, names = ['user_id','rating','date','item_id'],header = None)

movie_db.head()
# movie_db['date'] = pd.to_datetime(movie_db['timestamp'].astype(int), unit='s')
# movie_db['date'] = pd.to_datetime(movie_db['date'],format="%m/%d/%Y").dt.strftime('%Y-%m-%d')
movie_db=movie_db[['user_id','item_id','rating','date']]



In [3]:
#movie_db['date'].nunique() # 2000 user, 13344 item , 1951 date

In [4]:
from keras import backend as K

def recall_m(Y_train, pred_train):
    true_positives = K.sum(K.round(K.clip(Y_train * pred_train, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(Y_train, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(Y_train, pred_train):
    true_positives = K.sum(K.round(K.clip(Y_train * pred_train, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(pred_train, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(Y_train, pred_train):
    precision = precision_m(Y_train, pred_train)
    recall = recall_m(Y_train, pred_train)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


#################


In [5]:
item_info=movie_db.groupby('item_id', as_index=False).rating.agg({np.mean, np.var})
item_info.reset_index(level=0, inplace=True)
target_size=100
selected_item_size=20
performance=pd.DataFrame(columns=['attack_model','attack size','filler size','test_or_train','accuracy','recall','F1'])

x=0.01 ## percentage of fillers for AOP attack
avg_rating=movie_db['rating'].mean() 
std_rating=movie_db['rating'].std() 
#selected_items to be chosen from highly popular items , items that are rated by most of users ( highest number of ratings)
#item_info1=
#movie_db.groupby('item_id', as_index=False)['rating'].mean()
attack_date=random.sample(list(movie_db['date'].unique()),1)
print('attack day is: ',attack_date)

attack day is:  ['2003-05-31']


In [6]:
#for attack_i in ('random', 'average', 'bandwagon','AOP'):
    #for filler_i in (0.05):#(0.01,0.03,0.05,0.07,0.09):
     #   
#for attacksize_j in (0.1,0.15,0.2):
attack_model='AOP'#attack_i
filler_size= 0.01#filler_i # from 1% to 10% which is 1% of items excluding target items
attack_size= 0.1
fake_db = pd.DataFrame( columns=['user_id', 'item_id', 'rating', 'date'])
fake_profile=[]
filler_db=pd.DataFrame(columns=['user_id', 'item_id', 'rating', 'date'])
target_db=pd.DataFrame(columns=['user_id', 'item_id', 'rating', 'date'])
selected_db=pd.DataFrame(columns=['user_id', 'item_id', 'rating', 'date'])
# defining fake profiles
for i in range (round(attack_size*movie_db['user_id'].nunique())):
    fake_profile.append(movie_db['user_id'].max()+(i+1))
print('number of fake profiles generated is: \n',round(attack_size*movie_db['user_id'].nunique()))

#defining target items
#30 items are choosen randomly out of 1682 items
target_items=random.sample(list(movie_db['item_id'].unique()),target_size)
print('number of target items selected is: \n',target_size)


#defining filler items
fillers = [x for x in list(movie_db['item_id']) if x not in target_items]
#randomly choosed filler items based on filler size
filler_items=random.sample(fillers,round(filler_size*len(set(fillers))))




#defining selected items
selected=movie_db.groupby('item_id', as_index=False)['rating'].count()
selected=selected[selected['rating']>round(0.4*movie_db['user_id'].nunique())]
selected_items=[x for x in selected['item_id'] if x not in target_items if x not in filler_items]
print('number of selected items  is: \n',len(set(selected_items)))

#insert ratings for filler items
filler_db.drop(filler_db.index, inplace=True)
for i in filler_items:
    if attack_model=='random':
        filler_db=filler_db.append(pd.DataFrame({'user_id': random.sample(fake_profile,1), 'item_id': i ,'rating':round(np.random.normal(avg_rating, std_rating),2),'date':[attack_date[0]]}), ignore_index=True)
    if attack_model=='average':
        filler_db=filler_db.append(pd.DataFrame({'user_id': random.sample(fake_profile,1), 'item_id': i ,'rating':item_info.loc[item_info['item_id']==i,'mean'],'date':[attack_date[0]]}), ignore_index=True)
    if attack_model=='bandwagon':
        filler_db=filler_db.append(pd.DataFrame({'user_id': random.sample(fake_profile,1), 'item_id': i ,'rating':round(np.random.normal(avg_rating, std_rating),2),'date':[attack_date[0]]}), ignore_index=True)


#insert ratings for target items
target_db.drop(target_db.index, inplace=True)
for i in target_items:
    #print(random.sample(fake_profile,1),i,5)
    target_db=target_db.append(pd.DataFrame({'user_id':random.sample(fake_profile,1),'item_id':i,'rating':5,'date':[attack_date[0]]}),ignore_index=True)

#insert rating for selected items
if attack_model=='bandwagon':
    for i in selected_items:
        selected_db=selected_db.append(pd.DataFrame({'user_id': random.sample(fake_profile,1), 'item_id': i ,'rating':5,'date':[attack_date[0]]}), ignore_index=True)


#fillers selection for AOP attacks

AOP_filler_db=pd.DataFrame(columns=['user_id', 'item_id', 'rating', 'date'])
AOP_fillers=movie_db.groupby('item_id', as_index=False)['rating'].count()
AOP_fillers=AOP_fillers.sort_values('rating',ascending=False)
AOP_fillers=AOP_fillers.head(int(len(AOP_fillers)*(x)))
AOP_fillers=[x for x in AOP_fillers['item_id']]


if attack_model=='AOP':
    for i in AOP_fillers:
        AOP_filler_db=AOP_filler_db.append(pd.DataFrame({'user_id':random.sample(fake_profile,1), 'item_id': i ,'rating':np.random.normal(item_info.loc[item_info['item_id']==i,'mean'],item_info.loc[item_info['item_id']==i,'var']),'date':[attack_date[0]]}), ignore_index=True)
    filler_db=AOP_filler_db

if attack_model=='AOP':
    print('number of filler items selected for AOP attack is: \n',round(len(set(AOP_fillers))))
else:
    print('number of filler items selected is: \n',round(filler_size*len(set(fillers))))

# to first create a complete dataframe and then reshape to array
injected_db=pd.DataFrame()
injected_db=injected_db.append(target_db)
injected_db=injected_db.append(filler_db)
if attack_model=='bandwagon':
    injected_db=injected_db.append(selected_db)

movie_db=movie_db.append(injected_db,sort=True)
movie_db.fillna(0, inplace=True)

iix_n = pd.MultiIndex.from_product([np.unique(movie_db.user_id), np.unique(movie_db.date)])
arr = (movie_db.pivot_table('rating', ['user_id', 'date'], 'item_id', aggfunc='sum')
         .reindex(iix_n,copy=False).to_numpy()
         .reshape(movie_db.user_id.nunique(),movie_db.date.nunique(),-1))
arr_y=[1  for i in arr[:2000,:,:]]
arr_y2=[0  for i in arr[2000:,:,:] ]########??????????? how to give target to them????
arr_y3=arr_y+arr_y2

inds = np.where(np.isnan(arr))
#Place column means in the indices. Align the arrays using take
arr[inds] = 0
X_train, X_test, Y_train, Y_test = train_test_split(arr,arr_y3,test_size=0.30, random_state=40)


number of fake profiles generated is: 
 200
number of target items selected is: 
 100
number of selected items  is: 
 2
number of filler items selected for AOP attack is: 
 133


MemoryError: 

In [ ]:

del(arr)
del(inds)

##model creation
n_users=X_train.shape[0]
n_days=X_train.shape[1]
n_items=X_train.shape[2]
#input_shape=(1,732,213,1682)
#input_shape=(213,)
timesteps = 1
data_dim=1951
kernel_size=4
filters=32


model = Sequential()
################
#(batch_size, channels, rows, cols)
#, data_format="channels_last"
# ba in kar mikone : 1,n_users,n_days, n_items
#va in 1,n_days,n_users,n_items
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape =(1,n_days,n_items,1)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(3,3))))
model.add(TimeDistributed(Dropout(0.5)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(32
               , return_sequences=False,
               input_shape=(timesteps, data_dim)
              )
         )
## change data shape, to one 2D for every day and time steps as number of days
####model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


model.summary()
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy',f1_m,precision_m, recall_m]) 
Y_train = to_categorical(Y_train) ############
Y_test = to_categorical(Y_test)
X_train = np.reshape(X_train, (X_train.shape[0],1,X_train.shape[1],  X_train.shape[2],1))
X_test = np.reshape(X_test, (X_test.shape[0],1,X_test.shape[1],  X_test.shape[2],1))

model.fit(X_train, Y_train, epochs=5,batch_size=20)

pred_train= model.predict(X_train, batch_size=128)
#scores = model.evaluate(X_train, Y_train, verbose=0)
# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(X_train, Y_train, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(accuracy, 1 - accuracy))   
print('Recall on training data: {}% \n Error on training data: {}'.format(recall, 1 - recall)) 
print('F1 score on training data: {}% \n Error on training data: {}'.format(f1_score, 1 - f1_score)) 

pred_test= model.predict(X_test)
loss2, accuracy2, f1_score2, precision2, recall2  = model.evaluate(X_test, Y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(accuracy2, 1 - accuracy2))
print('Recall on test data: {}% \n Error on test data: {}'.format(recall2, 1 - recall2)) 
print('F1 score on test data: {}% \n Error on test data: {}'.format(f1_score2, 1 - f1_score2)) 
pred_train2=np.argmax(pred_train, axis=1)
#pred_train2 = np.round(pred_train)

Y_train2=np.argmax(Y_train, axis=1)
#Y_train2 = np.round(Y_train)

pred_test2=np.argmax(pred_test, axis=1)
#pred_test2 = np.round(pred_test)

Y_test2=np.argmax(Y_test, axis=1)
#Y_test2 = np.round(Y_test)
performance=performance.append(pd.DataFrame({'attack_model': [attack_model],'attack size':[attack_size],'filler size':[filler_size], 'test_or_train': 'train' ,'accuracy':[accuracy],'recall':[recall], 'F1':[f1_score]}), ignore_index=True)
performance=performance.append(pd.DataFrame({'attack_model': [attack_model],'attack size':[attack_size],'filler size':[filler_size], 'test_or_train': 'test' ,'accuracy':[accuracy2],'recall':[recall2], 'F1':[f1_score2]}), ignore_index=True)
gc.collect()



In [ ]:
#performance.to_csv("C:/Users/ebrah/Desktop/Ryerson/research1/performance/hybrid/inputmethod1.csv",mode='a',sep=',')
end_time = datetime.now()
print("started at: ",start_time)
print("ended at: ",end_time)

In [ ]:
performance